In [1]:
import torch
import torch.nn as nn
from fem.data import *
from fem.Poisson import Poisson
from fem.grf import GRF2d
import numpy as np
device = torch.device("cpu")


In [2]:
h = 1/64
n = int(1/h)
poisson = Poisson(h=h, quadrature_order=1, dtype=torch.float)
pde = GRF2d(n*4, alpha=2, tau=10, device=device, double=False)


In [3]:
poisson._assemble(pde)
poisson.solve()
uh_dir = poisson.get_u().cpu().numpy()

uI = pde.solution(pde.source.squeeze(1)).unsqueeze(1)
uI = F.interpolate(uI, size=(n+1, n+1),
                  mode='bilinear',
                  align_corners=True)
uI = uI.view(-1).numpy()

In [5]:
optimizer = torch.optim.LBFGS(poisson.parameters(), lr=1)
num_iter = 1
energies = []
# re-init
poisson = Poisson(h=h, quadrature_order=1, dtype=torch.float)
poisson._assemble(pde)
b = poisson.b_int

for k in range(num_iter):
    # flux = poisson.forward(poisson.u)

    def closure():
        optimizer.zero_grad()
        loss = poisson.energy(poisson.u, b)
        loss.backward(retain_graph=True)
        return loss

    optimizer.step(closure)

    with torch.no_grad():
        loss_val = poisson.energy(poisson.u, b)
        print(f"\n{k+1}-th iter")
        print(f"energy: \t {loss_val.item():.4e} ")
        energies.append(loss_val)
        uh = poisson.get_u().cpu().numpy()
        errL2 = np.linalg.norm(uh_dir - uh) * h
        errLinf = np.abs(uh_dir - uh).max()
        print(f"L2 error: \t {errL2:.4e} ")
        print(f"Linf error: \t {errLinf:.7f} ")



1-th iter
energy: 	 0.0000e+00 
L2 error: 	 3.0084e-03 
Linf error: 	 0.0083640 
